In [147]:
from surgeon_recording.reader import Reader
from os.path import join
import csv
import pandas as pd
import numpy as np
from tabulate import tabulate
import scipy.interpolate as inter
from pyquaternion import Quaternion
from numpy import linalg as LA
import plotly.graph_objects as go
import os
import scipy.ndimage
import scipy.signal
import matplotlib.pyplot as plt
import matplotlib
from plotly.subplots import make_subplots
from plotly import subplots
import matplotlib.lines


reader = Reader()

## READ DATA

In [2]:
path_to_data_folder = '/home/anaelle/git/surgeon_recording/data_paper'                 #lasa computer

subject_list = ['S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12']
segment_list = ['500','510','530','540','541','560']

sensors = ['optitrack']


tools_tweezer = ['tweezer_x', 'tweezer_y', 'tweezer_z']
tools_scissors = ['scissors_x', 'scissors_y', 'scissors_z'] 
tools_verif_holder = ['needle_holder2_x', 'needle_holder2_y', 'needle_holder2_z']


data_segments = {}
mean_segments = {}

run_list = {}
headers = {}
file_list = {}


# List of run we take for analysis, for each subject
run_list = {}
for subject in subject_list:
    run_list[subject] = {}

run_list['S2'] = ['R1','R3','R4']
run_list['S3'] = ['R2','R3','R4']
run_list['S4'] = ['R1','R3','R4']
run_list['S5'] = ['R3','R4','R5']
run_list['S6'] = ['R1','R3','R5']
run_list['S7'] = ['R1','R3','R4']
run_list['S8'] = ['R1','R3','R4']
run_list['S9'] = ['R2','R3','R5']
run_list['S10'] = ['R1','R2','R4']
run_list['S11'] = ['R1','R3','R4']
run_list['S12'] = ['R1','R2','R3']


# Create structures
for subject in subject_list:
    data_segments[subject] = {}
    mean_segments[subject]= {}
    file_list[subject] = {}
        
    for segment in segment_list:
        data_segments[subject][segment]= {}
        file_list[subject][segment] = {}
        mean_segments[subject][segment]= {}
                    
        for run in run_list[subject]:
            data_segments[subject][segment][run] = {}
            mean_segments[subject][segment][run]= {}
            file_list[subject][segment][run] = {}

            for sensor in sensors:
                data_segments[subject][segment][run][sensor]= {}
                mean_segments[subject][segment][run][sensor]= []
                file_list[subject][segment][run][sensor] = []


for sensor in sensors:
    headers[sensor] = []
                
for subject in subject_list:   
        for sensor in sensors:
            for segment in segment_list:
                for run in run_list[subject]:
                    data_folder = join('..', 'data_paper','OPTITRACK', subject, run )
                    file_list[subject][segment][run][sensor] = join(data_folder, 'segment_nb_' + segment + '_' + sensor +'.csv')

                    file = open(file_list[subject][segment][run][sensor])
                    reader = csv.DictReader(file)
                    headers[sensor] = reader.fieldnames

                    tools = headers[sensor][3:-1]

                    data = pd.read_csv(file_list[subject][segment][run][sensor])

                    for tool in tools:
                        data_segments[subject][segment][run][sensor][tool]= data[tool]
                        data_segments[subject][segment][run][sensor]['rel_time'] = data['relative_time']              
                    

                            

# DATA OPTITRACK

## REVIEW DATA

### Remove 0 values (loss of tracking)

In [3]:

segment_list_optitrack = ['500','510','530','540','541']
segment_new_optitrack = ['2','3','4','5','6']


tools_position = ['tweezer_x', 'tweezer_y', 'tweezer_z', 'needle_holder2_x', 'needle_holder2_y', 'needle_holder2_z']
tools_quaternions = ['tweezer_qx', 'tweezer_qy', 'tweezer_qz', 'tweezer_qw', 'needle_holder2_qx', 'needle_holder2_qy', 'needle_holder2_qz', 'needle_holder2_qw']

#tools = ['tweezer_x', 'tweezer_y', 'tweezer_z', 'tweezer_qx', 'tweezer_qy', 'tweezer_qz', 'tweezer_qw', 'scissors_x', 'scissors_y', 'scissors_z', 'scissors_qx', 'scissors_qy', 'scissors_qz', 'scissors_qw', 'needle_holder2_x', 'needle_holder2_y', 'needle_holder2_z', 'needle_holder2_qx', 'needle_holder2_qy', 'needle_holder2_qz', 'needle_holder2_qw']
tools = ['tweezer_x', 'tweezer_y', 'tweezer_z', 'tweezer_qx', 'tweezer_qy', 'tweezer_qz', 'tweezer_qw', 'needle_holder2_x', 'needle_holder2_y', 'needle_holder2_z', 'needle_holder2_qx', 'needle_holder2_qy', 'needle_holder2_qz', 'needle_holder2_qw']


sensor = 'optitrack'
optitrack_segment_subject = {}
optitrack_non_zero= {}
optitrack_non_zero_time = {}
for subject in subject_list:   
    optitrack_segment_subject[subject] = {}
    optitrack_non_zero[subject]= {}
    optitrack_non_zero_time[subject] = {}
    for segment in segment_list:
        optitrack_segment_subject[subject][segment] = {}
        optitrack_non_zero[subject][segment]= {}
        optitrack_non_zero_time[subject][segment] = {}
        for run in run_list[subject]:
            optitrack_segment_subject[subject][segment][run] = {}
            optitrack_non_zero[subject][segment][run]= {}
            optitrack_non_zero_time[subject][segment][run] = {}
            for tool in tools:  
                optitrack_segment_subject[subject][segment][run][tool]=[]
                optitrack_non_zero[subject][segment][run][tool] = []
                optitrack_non_zero_time[subject][segment][run] [tool] = []


#remove 0 values from loss of tracking FOR computations BUT should use vectors with the zeroes to keep correspondance of time with data
for subject in subject_list:   
    for segment in segment_list:
        for run in run_list[subject]:
            for tool in tools:
                time_current = data_segments[subject][segment][run][sensor]['rel_time']        
                data_current = data_segments[subject][segment][run][sensor][tool]
                
                optitrack_segment_subject[subject][segment][run][tool] = data_current
                optitrack_non_zero[subject][segment][run][tool] = data_current[data_current != 0]
                optitrack_non_zero_time[subject][segment][run][tool] = time_current[data_current != 0]



### Interpolation data top

In [4]:
Interpole = {}
time_vector_interp = {}
interpolated_data = {}
for subject in subject_list:   
    Interpole[subject] = {}
    time_vector_interp[subject]= {}
    interpolated_data[subject] = {}
    for segment in segment_list:
        Interpole[subject][segment] = {}
        time_vector_interp[subject][segment]= {}
        interpolated_data[subject][segment] = {}
        
        for run in run_list[subject]:
            Interpole[subject][segment][run] = {}
            time_vector_interp[subject][segment][run]= {}
            interpolated_data[subject][segment][run] = {}
            for tool in tools:
                Interpole[subject][segment][run][tool] = {}
                time_vector_interp[subject][segment][run][tool]= {}
                interpolated_data[subject][segment][run][tool] = {}


for subject in subject_list:   
    for segment in segment_list:
        for run in run_list[subject]:
            for tool in tools:

                data = np.array(optitrack_non_zero[subject][segment][run][tool])
                time = np.array(optitrack_non_zero_time[subject][segment][run][tool])
                Interpole[subject][segment][run][tool] = inter.interp1d(time, data )
                time_step = 1/120
                time_vector_interp[subject][segment][run][tool] = np.arange(time.min(),time.max()-time_step,time_step )
                interpolated_data[subject][segment][run][tool] = Interpole[subject][segment][run][tool](time_vector_interp[subject][segment][run][tool])
      

### Transformation to have tip position

In [5]:
# hardcoded tool and tooltip rotation and position in form [qw,qx,qy,qz,tx,ty,tz,1]
#  Quaternion((w, x, y, z)) function: Creates a unit quaternion 1 + 0i + 0j + 0k
# approximation for the 1st set of recorded data (not exact as fell several times + position measured afterwards, the marker was not placed exactly at same location)

tools_name = ["tweezer", "needle_holder2"]
tools_transf = {
    "tweezer": {
        "marker_tip": [-0.14002110139417,0.011598759188847,0.914789185044361,-0.378701883396705,0.203922977186312,0.225921215462611,0.337081560202788,1],     # the one from the fixed marker for tip measurement
        "tip":  [-0.659606319391634,0.047673735107731,-0.082923699619772,-0.520023288973384,0.173957278833967,0.147784077313054,0.464396291508238,1]    # marker to rpz tip
        
    },

    "needle_holder2":  {
        "marker_tip": [-0.921827204331451,0.195545852165725,0.143764196798494,-0.302187299435028,0.181443717514124,0.300805392655367,0.335198482109228,1],
        "tip":  [-0.864763969868174,-0.013123148775895,-0.07854127306968,-0.49579974858757,0.170293133709981,0.248165722222222,0.464240151600753,1]

    }       
}


for tool in tools_name:    
    marker_tip_quaternion = Quaternion(tools_transf[tool]["marker_tip"][:4])  #Return homogeneous rotation matrix from quaternion. = on calcule la matrice de rotation qui correspond aux quaternions du tool (chiffres en haut) #ATTENTION: normalise le quaternion a 1 donc pas exactemetn meme valeurs
    T_marker_tip_fixed = marker_tip_quaternion.transformation_matrix         # deja en 4*4 matrix
    
    marker_tip_translation_vector = np.array((tools_transf[tool]["marker_tip"][-4:])) #Return matrix to translate by direction vector = on calcule la matrice de translation qui correspond aux positions du tool (chiffres en haut)
    T_marker_tip_fixed[:,-1] = marker_tip_translation_vector       #matrice de transformation pour passer de mon objet au centre du reference frame à sa position actuelle (marker de l'outils)
        

    tip_quaternion = Quaternion(tools_transf[tool]["tip"][:4]) #matrice de rotation des quaternions du tip #ATTENTION: normalise le quaternion a 1 donc pas exactemetn meme valeurs
    T_tip = tip_quaternion.transformation_matrix
  
    tip_translation_vector = np.array((tools_transf[tool]["tip"][-4:])) #-4 car 4 dernieres valeurs
    T_tip[:,-1] = tip_translation_vector       #matrice de transformation pour passer de mon objet au centre du reference frame à sa position actuelle (tip de l'outils)
    
    T_marker_tip_fixed_inv = LA.inv(T_marker_tip_fixed)  
    
## ORIGIN TO TIP, BASED ON MARKER POSITION
     
    T_marker_tip = T_marker_tip_fixed_inv.dot(T_tip) #Matrice mTtip = marker to tip
    q_marker_tip = Quaternion(matrix = T_marker_tip)
    

In [117]:
# need to put them in the format to compute oTmarker
translation_vector = {}
quaternion_vector = {}
position_vector_tip = {}
position_vector_top = {}
vector_top_to_tip = {}
pos_x = {}
pos_y = {}
pos_z = {}
pos_x_tip = {}
pos_y_tip = {}
pos_z_tip = {}
pos_x_tip_outlier = {}
pos_y_tip_outlier = {}
pos_z_tip_outlier = {} 
for subject in subject_list:
    translation_vector[subject] = {}
    quaternion_vector[subject] = {}
    position_vector_tip[subject] = {}
    position_vector_top[subject] = {}

    vector_top_to_tip[subject] = {}
    pos_x[subject]= {}
    pos_y[subject]= {}
    pos_z[subject]= {}

    pos_x_tip[subject]= {}
    pos_y_tip[subject]= {}
    pos_z_tip[subject]= {}
    pos_x_tip_outlier[subject]= {}
    pos_y_tip_outlier[subject]= {}
    pos_z_tip_outlier[subject]= {}
    for segment in segment_list:
        translation_vector[subject][segment] = {}
        quaternion_vector[subject][segment] = {}
        position_vector_tip[subject][segment] = {}
        position_vector_top[subject][segment] = {}

        vector_top_to_tip[subject][segment] = {}
        pos_x[subject][segment]= {}
        pos_y[subject][segment]= {}
        pos_z[subject][segment]= {}
        pos_x_tip[subject][segment]= {}
        pos_y_tip[subject][segment]= {}
        pos_z_tip[subject][segment]= {}        
        pos_x_tip_outlier[subject][segment]= {}
        pos_y_tip_outlier[subject][segment]= {}
        pos_z_tip_outlier[subject][segment]= {}         
        for run in run_list[subject]:
            translation_vector[subject][segment][run] = {}
            quaternion_vector[subject][segment][run] = {}
            position_vector_tip[subject][segment][run] = {}
            position_vector_top[subject][segment][run] = {}
            vector_top_to_tip[subject][segment][run] = {}
            pos_x[subject][segment][run]= {}
            pos_y[subject][segment][run]= {}
            pos_z[subject][segment][run]= {}
            pos_x_tip[subject][segment][run]= {}
            pos_y_tip[subject][segment][run]= {}
            pos_z_tip[subject][segment][run]= {}
            pos_x_tip_outlier[subject][segment][run]= {}
            pos_y_tip_outlier[subject][segment][run]= {}
            pos_z_tip_outlier[subject][segment][run]= {}            
            for tool in tools_name:
                translation_vector[subject][segment][run][tool] = []
                quaternion_vector[subject][segment][run][tool]  = []
                position_vector_tip[subject][segment][run][tool]  = []
                position_vector_top[subject][segment][run][tool]  = []
                vector_top_to_tip[subject][segment][run][tool] = []
                pos_x[subject][segment][run][tool] = []
                pos_y[subject][segment][run][tool] = []
                pos_z[subject][segment][run][tool] = []
                pos_x_tip[subject][segment][run][tool] = []
                pos_y_tip[subject][segment][run][tool] = []
                pos_z_tip[subject][segment][run][tool] = []
                pos_x_tip_outlier[subject][segment][run][tool] = []
                pos_y_tip_outlier[subject][segment][run][tool] = []
                pos_z_tip_outlier[subject][segment][run][tool] = []
                
                #for i in range(len(interpolated_data[subject][segment][run][tool + '_' + 'x'])):
                for i in range(len(optitrack_non_zero[subject][segment][run][tool + '_' + 'x'])):

                    #current_trans_vect = np.array([np.array(interpolated_data[subject][segment][run][tool + '_' + 'x'])[i], np.array(interpolated_data[subject][segment][run][tool + '_' + 'y'])[i], np.array(interpolated_data[subject][segment][run][tool + '_' + 'z'])[i],1])
                    # with interpolated data
                    current_trans_vect = np.array([np.array(optitrack_non_zero[subject][segment][run][tool + '_' + 'x'])[i], np.array(optitrack_non_zero[subject][segment][run][tool + '_' + 'y'])[i], np.array(optitrack_non_zero[subject][segment][run][tool + '_' + 'z'])[i],1])
                    #with raw data
                    
                    translation_vector[subject][segment][run][tool].append(current_trans_vect)
                    #current_quaternion = np.array([np.array(interpolated_data[subject][segment][run][tool + '_' + 'qw'])[i], np.array(interpolated_data[subject][segment][run][tool + '_' + 'qx'])[i], np.array(interpolated_data[subject][segment][run][tool + '_' + 'qy'])[i], np.array(interpolated_data[subject][segment][run][tool + '_' + 'qz'])[i]])
                    #with interpolated data
                    current_quaternion = np.array([np.array(optitrack_non_zero[subject][segment][run][tool + '_' + 'qw'])[i], np.array(optitrack_non_zero[subject][segment][run][tool + '_' + 'qx'])[i], np.array(optitrack_non_zero[subject][segment][run][tool + '_' + 'qy'])[i], np.array(optitrack_non_zero[subject][segment][run][tool + '_' + 'qz'])[i]])
                    #with raw data
                    #ajoute a la suite pour une action, 1 sujet, un outils et un sgment le vecteur ieme vecteur x,y,z 
                    q = Quaternion(current_quaternion)
                    quaternion_vector[subject][segment][run][tool].append(current_quaternion)                       
                   
                    T_marker = q.transformation_matrix        
                    T_marker[:,-1] = current_trans_vect
                    
                    tmp_pos_tip = T_marker.dot(T_marker_tip)[0:3,-1]
                    tmp_pos_top = current_trans_vect[0:3]
                    
                    position_vector_tip[subject][segment][run][tool].append(tmp_pos_tip) #prend le vecteur de translation de la matrice obtenue qui correspond a la position par rapport au referentiel
                    position_vector_top[subject][segment][run][tool].append(tmp_pos_top)
                    
                    tmp_top_to_tip = np.subtract(tmp_pos_tip, tmp_pos_top)
                    
                    vector_top_to_tip[subject][segment][run][tool].append(tmp_top_to_tip) #format x,y,y
                
                #pos_x[subject][segment][run][tool] = [item[0] for item in vector_top_to_tip[subject][segment][run][tool]]  #que les x
                #pos_y[subject][segment][run][tool] = [item[1] for item in vector_top_to_tip[subject][segment][run][tool]]  
                #pos_z[subject][segment][run][tool] = [item[2] for item in vector_top_to_tip[subject][segment][run][tool]]
                
                pos_x[subject][segment][run][tool] = [item[0] for item in position_vector_top[subject][segment][run][tool]]  #que les x
                pos_y[subject][segment][run][tool] = [item[1] for item in position_vector_top[subject][segment][run][tool]]  
                pos_z[subject][segment][run][tool] = [item[2] for item in position_vector_top[subject][segment][run][tool]]
                
                pos_x_tip[subject][segment][run][tool] = [item[0] for item in position_vector_tip[subject][segment][run][tool]]  #que les x
                pos_y_tip[subject][segment][run][tool] = [item[1] for item in position_vector_tip[subject][segment][run][tool]]  
                pos_z_tip[subject][segment][run][tool] = [item[2] for item in position_vector_tip[subject][segment][run][tool]]
                
                
            

### REMOVE OUTLIERS

In [118]:
time_pos_x_tip_outlier={}
time_pos_y_tip_outlier={}
time_pos_z_tip_outlier={}
for subject in subject_list:
    time_pos_x_tip_outlier[subject]={}
    time_pos_y_tip_outlier[subject]={}
    time_pos_z_tip_outlier[subject]={}
    for segment in segment_list:
        time_pos_x_tip_outlier[subject][segment]={}
        time_pos_y_tip_outlier[subject][segment]={}
        time_pos_z_tip_outlier[subject][segment]={}
        for run in run_list[subject]:
            time_pos_x_tip_outlier[subject][segment][run]={}
            time_pos_y_tip_outlier[subject][segment][run]={}
            time_pos_z_tip_outlier[subject][segment][run]={}
            for tool in tools_name:
                time_pos_x_tip_outlier[subject][segment][run][tool] = []
                time_pos_y_tip_outlier[subject][segment][run][tool] = []
                time_pos_z_tip_outlier[subject][segment][run][tool] = []


#for y: threshold ok
for subject in subject_list:
    for segment in segment_list:
        for run in run_list[subject]:
            for tool in tools_name:
                pos_x_tip_outlier[subject][segment][run][tool] = pd.DataFrame(pos_x_tip[subject][segment][run][tool])
                pos_z_tip_outlier[subject][segment][run][tool] = pd.DataFrame(pos_z_tip[subject][segment][run][tool])
                time_pos_x_tip_outlier[subject][segment][run][tool] = pd.DataFrame(list(optitrack_non_zero_time[subject][segment][run][tool+ '_' + 'x']))
                time_pos_z_tip_outlier[subject][segment][run][tool] = pd.DataFrame(list(optitrack_non_zero_time[subject][segment][run][tool+ '_' + 'z']))
                
                data_y = pd.DataFrame(pos_y_tip[subject][segment][run][tool])
                pos_y_tip_outlier[subject][segment][run][tool] = data_y[data_y<0.15]
                time = pd.DataFrame(list(optitrack_non_zero_time[subject][segment][run][tool+ '_' + 'y']))
                time_pos_y_tip_outlier[subject][segment][run][tool] = time[data_y<0.2]


#for x: specifically
for segment in ['500', '560']:
    data_x = pos_x_tip_outlier['S2'][segment]['R3']['needle_holder2']
    time_x = time_pos_x_tip_outlier['S2'][segment]['R3']['needle_holder2']
    pos_x_tip_outlier['S2'][segment]['R3']['needle_holder2'] = data_x[data_x<0.6]
    time_pos_x_tip_outlier['S2'][segment]['R3'][tool] = time_x[data_x<0.6]

for segment in ['510', '560']:
    data_x = pos_x_tip_outlier['S9'][segment]['R3']['needle_holder2']
    time_x = time_pos_x_tip_outlier['S9'][segment]['R3']['needle_holder2']
    pos_x_tip_outlier['S9'][segment]['R3']['needle_holder2'] = data_x[data_x<0.4]
    time_pos_x_tip_outlier['S9'][segment]['R3']['needle_holder2'] = time_x[data_x<0.4]
    
for segment in ['540', '560']:
    data_x = pos_x_tip_outlier['S12'][segment]['R3']['needle_holder2']
    time_x = time_pos_x_tip_outlier['S12'][segment]['R3']['needle_holder2']
    pos_x_tip_outlier['S12'][segment]['R3']['needle_holder2'] = data_x[data_x<0.5]
    time_pos_x_tip_outlier['S12'][segment]['R3']['needle_holder2'] = time_x[data_x<0.5]
    
for segment in ['510']:
    data_x = pos_x_tip_outlier['S9'][segment]['R2']['tweezer']
    time_x = time_pos_x_tip_outlier['S9'][segment]['R2']['tweezer']
    pos_x_tip_outlier['S9'][segment]['R2']['tweezer'] = data_x[data_x>0.42]
    time_pos_x_tip_outlier['S9'][segment]['R2']['tweezer'] = time_x[data_x>0.42]

for segment in ['510', '560']:
    data_x = pos_x_tip_outlier['S9'][segment]['R3']['tweezer']
    time_x = time_pos_x_tip_outlier['S9'][segment]['R3']['tweezer']
    pos_x_tip_outlier['S9'][segment]['R3']['tweezer'] = data_x[data_x>0.28]
    time_pos_x_tip_outlier['S9'][segment]['R3']['tweezer'] = time_x[data_x>0.28] 
    
for segment in ['560']:
    data_x = pos_x_tip_outlier['S9'][segment]['R5']['tweezer']
    time_x = time_pos_x_tip_outlier['S9'][segment]['R5']['tweezer']
    pos_x_tip_outlier['S9'][segment]['R5']['tweezer'] = data_x[data_x>0.28]
    time_pos_x_tip_outlier['S9'][segment]['R5']['tweezer'] = time_x[data_x>0.28] 
    
# for z
for segment in ['510', '560']:
    data_z = pos_z_tip_outlier['S9'][segment]['R3']['needle_holder2']
    time_z = time_pos_z_tip_outlier['S9'][segment]['R3']['needle_holder2']
    pos_z_tip_outlier['S9'][segment]['R3']['needle_holder2'] = data_z[data_z>0.02]
    time_pos_z_tip_outlier['S9'][segment]['R3'][tool] = time_z[data_z>0.02]

for segment in ['540', '560']:
    data_z = pos_z_tip_outlier['S12'][segment]['R3']['needle_holder2']
    time_z = time_pos_z_tip_outlier['S12'][segment]['R3']['needle_holder2']
    pos_z_tip_outlier['S12'][segment]['R3']['needle_holder2'] = data_z[data_z>0.05]
    time_pos_z_tip_outlier['S12'][segment]['R3'][tool] = time_z[data_z>0.05]
    
for segment in ['540', '560']:
    data_z = pos_z_tip_outlier['S2'][segment]['R1']['tweezer']
    time_z = time_pos_z_tip_outlier['S2'][segment]['R1']['tweezer']
    pos_z_tip_outlier['S2'][segment]['R1']['tweezer'] = data_z[data_x>0.18]
    time_pos_z_tip_outlier['S2'][segment]['R1']['tweezer'] = time_z[data_x>0.18] 
    
for segment in ['530', '560']:
    data_z = pos_z_tip_outlier['S7'][segment]['R1']['tweezer']
    time_z = time_pos_z_tip_outlier['S7'][segment]['R1']['tweezer']
    pos_z_tip_outlier['S7'][segment]['R1']['tweezer'] = data_z[data_x>0.07]
    time_pos_z_tip_outlier['S7'][segment]['R1']['tweezer'] = time_z[data_x>0.07] 
    
for segment in ['500']:
    data_z = pos_z_tip_outlier['S8'][segment]['R3']['tweezer']
    time_z = time_pos_z_tip_outlier['S8'][segment]['R3']['tweezer']
    pos_z_tip_outlier['S8'][segment]['R3']['tweezer'] = data_z[data_x>0.07]
    time_pos_z_tip_outlier['S8'][segment]['R3']['tweezer'] = time_z[data_x>0.07] 
    
for segment in ['510', '560']:
    data_z = pos_z_tip_outlier['S9'][segment]['R3']['tweezer']
    time_z = time_pos_z_tip_outlier['S9'][segment]['R3']['tweezer']
    pos_z_tip_outlier['S9'][segment]['R3']['tweezer'] = data_z[data_x>0.18]
    time_pos_z_tip_outlier['S9'][segment]['R3']['tweezer'] = time_z[data_x>0.18] 

['tweezer', 'needle_holder2']


### interpolation data tip

In [121]:

interpolated_data_tip_y = {}
interpolated_data_tip_x = {}
interpolated_data_tip_z = {}

for subject in subject_list:   
    interpolated_data_tip_y[subject] = {}
    interpolated_data_tip_x[subject] = {}
    interpolated_data_tip_z[subject] = {}

    for segment in segment_list:
        interpolated_data_tip_y[subject][segment] = {}
        interpolated_data_tip_x[subject][segment] = {}
        interpolated_data_tip_z[subject][segment] = {}
        for run in run_list[subject]:
            interpolated_data_tip_y[subject][segment][run] = {}
            interpolated_data_tip_x[subject][segment][run] = {}
            interpolated_data_tip_z[subject][segment][run] = {}

            for tool in tools_name:
                interpolated_data_tip_x[subject][segment][run][tool] = {}
                interpolated_data_tip_y[subject][segment][run][tool] = {}
                interpolated_data_tip_z[subject][segment][run][tool] = {}


for subject in subject_list:   
    for segment in segment_list:
        for run in run_list[subject]:
            for tool in tools_name:
                data_y = list(pos_y_tip_outlier[subject][segment][run][tool][0])
                data_x = list(pos_x_tip_outlier[subject][segment][run][tool][0])
                data_z = list(pos_z_tip_outlier[subject][segment][run][tool][0])
                
                time = np.array(optitrack_non_zero_time[subject][segment][run][tool+ '_' + 'x'])
                Interpole_y = inter.interp1d(list(time_pos_y_tip_outlier[subject][segment][run][tool][0]), list(data_y) )
                Interpole_x = inter.interp1d(list(time_pos_x_tip_outlier[subject][segment][run][tool][0]), list(data_x) )
                Interpole_z = inter.interp1d(list(time_pos_z_tip_outlier[subject][segment][run][tool][0]), list(data_z) )
                
                time_step = 1/120
                time_vector = np.arange(time.min(),time.max()-time_step,time_step )
                interpolated_data_tip_x[subject][segment][run][tool] = Interpole_x(time_vector)
                interpolated_data_tip_y[subject][segment][run][tool] = Interpole_y(time_vector)
                interpolated_data_tip_z[subject][segment][run][tool] = Interpole_z(time_vector)


### Median Filter

In [122]:

pos_x_tip_filter = {}
pos_y_tip_filter = {}
pos_z_tip_filter = {} 
for subject in subject_list:
    pos_x_tip_filter[subject]= {}
    pos_y_tip_filter[subject]= {}
    pos_z_tip_filter[subject]= {}
    for segment in segment_list:     
        pos_x_tip_filter[subject][segment]= {}
        pos_y_tip_filter[subject][segment]= {}
        pos_z_tip_filter[subject][segment]= {}         
        for run in run_list[subject]:
            pos_x_tip_filter[subject][segment][run]= {}
            pos_y_tip_filter[subject][segment][run]= {}
            pos_z_tip_filter[subject][segment][run]= {}            
            for tool in tools_name:
                pos_x_tip_filter[subject][segment][run][tool] = []
                pos_y_tip_filter[subject][segment][run][tool] = []
                pos_z_tip_filter[subject][segment][run][tool] = []              


                pos_x_tip_filter[subject][segment][run][tool] = scipy.ndimage.median_filter(interpolated_data_tip_x[subject][segment][run][tool] , size=14)
                pos_y_tip_filter[subject][segment][run][tool] = scipy.ndimage.median_filter(interpolated_data_tip_y[subject][segment][run][tool] , size=14, footprint=None, output=None, mode='reflect', cval=0.0, origin=0)
                pos_z_tip_filter[subject][segment][run][tool] = scipy.ndimage.median_filter(interpolated_data_tip_z[subject][segment][run][tool] , size=14, footprint=None, output=None, mode='reflect', cval=0.0, origin=0)
           

### Derivative estimator (2nd filter and derivative)

In [191]:

speed_x_tip_estimator= {}
speed_y_tip_estimator = {}
speed_z_tip_estimator = {} 
acc_x_tip_estimator= {}
acc_y_tip_estimator = {}
acc_z_tip_estimator = {} 
jerk_x_tip_estimator= {}
jerk_y_tip_estimator = {}
jerk_z_tip_estimator = {} 

speed_x_tip = {} 
speed_y_tip = {} 
speed_z_tip = {} 
acc_x_tip= {}
acc_y_tip = {}
acc_z_tip = {} 
jerk_x_tip= {}
jerk_y_tip = {}
jerk_z_tip = {} 
for subject in subject_list:
    speed_x_tip_estimator[subject]= {}
    speed_y_tip_estimator[subject]= {}
    speed_z_tip_estimator [subject]= {}
    acc_x_tip_estimator[subject]= {}
    acc_y_tip_estimator[subject]= {}
    acc_z_tip_estimator [subject]= {}
    jerk_x_tip_estimator[subject]= {}
    jerk_y_tip_estimator[subject]= {}
    jerk_z_tip_estimator [subject]= {}
    speed_x_tip[subject]= {}
    speed_y_tip[subject]= {}
    speed_z_tip[subject]= {}
    acc_x_tip[subject]= {}
    acc_y_tip[subject]= {}
    acc_z_tip[subject]= {}
    jerk_x_tip[subject]= {}
    jerk_y_tip[subject]= {}
    jerk_z_tip[subject]= {}
    for segment in segment_list:     
        speed_x_tip_estimator[subject][segment]= {}
        speed_y_tip_estimator[subject][segment]= {}
        speed_z_tip_estimator [subject][segment]= {}      
        acc_x_tip_estimator[subject][segment]= {}
        acc_y_tip_estimator[subject][segment]= {}
        acc_z_tip_estimator [subject][segment]= {}   
        jerk_x_tip_estimator[subject][segment]= {}
        jerk_y_tip_estimator[subject][segment]= {}
        jerk_z_tip_estimator [subject][segment]= {}  
        speed_x_tip[subject][segment]= {}  
        speed_y_tip[subject][segment]= {} 
        speed_z_tip[subject][segment]= {} 
        acc_x_tip[subject][segment]= {}
        acc_y_tip[subject][segment]= {}
        acc_z_tip[subject][segment]= {}   
        jerk_x_tip[subject][segment]= {}
        jerk_y_tip[subject][segment]= {}
        jerk_z_tip[subject][segment]= {}  
        for run in run_list[subject]:
            speed_x_tip_estimator[subject][segment][run]= {}
            speed_y_tip_estimator[subject][segment][run]= {}
            speed_z_tip_estimator [subject][segment][run]= {}
            acc_x_tip_estimator[subject][segment][run]= {}
            acc_y_tip_estimator[subject][segment][run]= {}
            acc_z_tip_estimator [subject][segment][run]= {}  
            jerk_x_tip_estimator[subject][segment][run]= {}
            jerk_y_tip_estimator[subject][segment][run]= {}
            jerk_z_tip_estimator [subject][segment][run]= {}  
            speed_x_tip [subject][segment][run]= {}  
            speed_y_tip [subject][segment][run]= {}
            speed_z_tip [subject][segment][run]= {}
            acc_x_tip[subject][segment][run]= {}
            acc_y_tip[subject][segment][run]= {}
            acc_z_tip[subject][segment][run]= {}  
            jerk_x_tip[subject][segment][run]= {}
            jerk_y_tip[subject][segment][run]= {}
            jerk_z_tip[subject][segment][run]= {}  
            for tool in tools_name:
                speed_x_tip_estimator[subject][segment][run][tool] = []
                speed_y_tip_estimator[subject][segment][run][tool] = []
                speed_z_tip_estimator [subject][segment][run][tool] = []     
                acc_x_tip_estimator[subject][segment][run][tool] = []
                acc_y_tip_estimator[subject][segment][run][tool] = []
                acc_z_tip_estimator [subject][segment][run][tool] = []
                jerk_x_tip_estimator[subject][segment][run][tool] = []
                jerk_y_tip_estimator[subject][segment][run][tool] = []
                jerk_z_tip_estimator [subject][segment][run][tool] = []
                speed_x_tip [subject][segment][run][tool] = []
                speed_y_tip [subject][segment][run][tool] = []
                speed_z_tip [subject][segment][run][tool] = []
                acc_x_tip[subject][segment][run][tool] = []
                acc_y_tip[subject][segment][run][tool] = []
                acc_z_tip[subject][segment][run][tool] = []
                jerk_x_tip[subject][segment][run][tool] = []
                jerk_y_tip[subject][segment][run][tool] = []
                jerk_z_tip[subject][segment][run][tool] = []
                
                window_length = 5
                polyorder = 3
                dt = 1/120
                speed_x_tip_estimator[subject][segment][run][tool] = scipy.signal.savgol_filter(pos_x_tip_filter[subject][segment][run][tool], window_length, polyorder, deriv=1, delta=dt, axis=- 1, mode='interp', cval=0.0)
                speed_y_tip_estimator[subject][segment][run][tool] = scipy.signal.savgol_filter(pos_y_tip_filter[subject][segment][run][tool], window_length, polyorder, deriv=1, delta=dt, axis=- 1, mode='interp', cval=0.0)
                speed_z_tip_estimator[subject][segment][run][tool] = scipy.signal.savgol_filter(pos_z_tip_filter[subject][segment][run][tool], window_length, polyorder, deriv=1, delta=dt, axis=- 1, mode='interp', cval=0.0)
                
                acc_x_tip_estimator[subject][segment][run][tool] = scipy.signal.savgol_filter(pos_x_tip_filter[subject][segment][run][tool], window_length, polyorder, deriv=2, delta=dt, axis=- 1, mode='interp', cval=0.0)
                acc_y_tip_estimator[subject][segment][run][tool] = scipy.signal.savgol_filter(pos_y_tip_filter[subject][segment][run][tool], window_length, polyorder, deriv=2, delta=dt, axis=- 1, mode='interp', cval=0.0)
                acc_z_tip_estimator[subject][segment][run][tool] = scipy.signal.savgol_filter(pos_z_tip_filter[subject][segment][run][tool], window_length, polyorder, deriv=2, delta=dt, axis=- 1, mode='interp', cval=0.0)
                
                jerk_x_tip_estimator[subject][segment][run][tool] = scipy.signal.savgol_filter(pos_x_tip_filter[subject][segment][run][tool], window_length,polyorder, deriv=3, delta=dt, axis=- 1, mode='interp', cval=0.0)
                jerk_y_tip_estimator[subject][segment][run][tool] = scipy.signal.savgol_filter(pos_y_tip_filter[subject][segment][run][tool], window_length, polyorder, deriv=3, delta=dt, axis=- 1, mode='interp', cval=0.0)
                jerk_z_tip_estimator[subject][segment][run][tool] = scipy.signal.savgol_filter(pos_z_tip_filter[subject][segment][run][tool], window_length, polyorder, deriv=3, delta=dt, axis=- 1, mode='interp', cval=0.0)
                
                data_current_x = pos_x_tip_filter[subject][segment][run][tool]
                data_current_y = pos_y_tip_filter[subject][segment][run][tool]
                data_current_z = pos_z_tip_filter[subject][segment][run][tool]
                time_current = time_vector_interp[subject][segment][run][tool + '_x']
                speed_x_tip[subject][segment][run][tool] = (data_current_x[1:-1] - data_current_x[0:-2])/(time_current[1:-1] - time_current[0:-2])
                speed_y_tip[subject][segment][run][tool] = (data_current_y[1:-1] - data_current_y[0:-2])/(time_current[1:-1] - time_current[0:-2])
                speed_z_tip[subject][segment][run][tool] = (data_current_z[1:-1] - data_current_z[0:-2])/(1/120)
                
                acc_x_tip[subject][segment][run][tool] = (speed_x_tip[subject][segment][run][tool][1:-1] - speed_x_tip[subject][segment][run][tool][0:-2])/(1/120)
                acc_y_tip[subject][segment][run][tool] = (speed_y_tip[subject][segment][run][tool][1:-1] - speed_y_tip[subject][segment][run][tool][0:-2])/(1/120)
                acc_z_tip[subject][segment][run][tool] = (speed_z_tip[subject][segment][run][tool][1:-1] - speed_z_tip[subject][segment][run][tool][0:-2])/(1/120)
                
                jerk_x_tip[subject][segment][run][tool] = (acc_x_tip[subject][segment][run][tool][1:-1] - acc_x_tip[subject][segment][run][tool][0:-2])/(1/120)
                jerk_y_tip[subject][segment][run][tool] = (acc_y_tip[subject][segment][run][tool][1:-1] - acc_y_tip[subject][segment][run][tool][0:-2])/(1/120)
                jerk_z_tip[subject][segment][run][tool] = (acc_z_tip[subject][segment][run][tool][1:-1] - acc_z_tip[subject][segment][run][tool][0:-2])/(1/120)
                
                

In [185]:
print((time_current[1:-1] - time_current[0:-2]))
print(1/120)

[0.00833333 0.00833333 0.00833333 ... 0.00833333 0.00833333 0.00833333]
0.008333333333333333


## plots to verify

In [ ]:
for subject in subject_list:   
    for segment in segment_list:
        for run in run_list[subject]:
            fig = go.Figure()
            for tool in ['tweezer_x', 'tweezer_y', 'tweezer_z','needle_holder2_x', 'needle_holder2_y', 'needle_holder2_z']:  
                time_current =  optitrack_non_zero_time[subject][segment][run][tool]

                data_current1 = np.array(optitrack_non_zero[subject][segment][run][tool])
                data_current2 = np.array(interpolated_data[subject][segment][run][tool])
                               
                fig.add_trace(go.Scatter(y=data_current2, x=np.array(time_vector_interp[subject][segment][run][tool] -data_segments[subject][segment][run][sensor]['rel_time'][0]),
                    mode='markers', name=tool+ "interp"))
                fig.add_trace(go.Scatter(y=data_current1, x=np.array((optitrack_non_zero_time[subject][segment][run][tool])-data_segments[subject][segment][run][sensor]['rel_time'][0]),
                    mode='markers', name=tool ))
            fig.update_layout(title='Subject: ' + subject + ', Run: ' + str(run) +', Segment:' + str(segment),
                xaxis_title='Time',
                yaxis_title='Optitrack data [m]',
                width=1000,
                height=500)
            #fig.show()
                       
            if not os.path.exists(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK', subject)):
                os.makedirs(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK', subject ))
            fig.write_image(join('..', 'data_paper',  'PLOTS' , 'OPTITRACK', subject, 'optitrack_position_INTERP_subject_'+ subject+ 'run'+ str(run) + '_segment' + segment  +'.png'))

In [ ]:
for subject in subject_list:   
    for segment in segment_list:
        for run in run_list[subject]:
            fig = go.Figure()
            for tool in ['tweezer_qx', 'tweezer_qy', 'tweezer_qz', 'tweezer_qw','needle_holder2_qx', 'needle_holder2_qy', 'needle_holder2_qz', 'needle_holder2_qw']:  
                time_current =  optitrack_non_zero_time[subject][segment][run][tool]

                data_current1 = np.array(optitrack_non_zero[subject][segment][run][tool])
                data_current2 = np.array(interpolated_data[subject][segment][run][tool])
                               
                fig.add_trace(go.Scatter(y=data_current2, x=np.array(time_vector_interp[subject][segment][run][tool] -data_segments[subject][segment][run][sensor]['rel_time'][0]),
                    mode='markers', name=tool+ "interp"))
                fig.add_trace(go.Scatter(y=data_current1, x=np.array((optitrack_non_zero_time[subject][segment][run][tool])-data_segments[subject][segment][run][sensor]['rel_time'][0]),
                    mode='markers', name=tool ))
            fig.update_layout(title='Subject: ' + subject + ', Run: ' + str(run) +', Segment:' + str(segment),
                xaxis_title='Time',
                yaxis_title='Optitrack data [m]',
                width=1000,
                height=500)
            #fig.show()
                       
            if not os.path.exists(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK', subject)):
                os.makedirs(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK', subject ))
            fig.write_image(join('..', 'data_paper',  'PLOTS' , 'OPTITRACK', subject, 'optitrack_quaternion_INTERP_subject_'+ subject+ 'run'+ str(run) + '_segment' + segment  +'.png'))

        

### plots data with filter/  interpolation / outliers...

In [ ]:
# plots signaux total
for subject in subject_list:   
    for segment in segment_list:
        for run in run_list[subject]:
            for tool in tools_name:
                time_current =  optitrack_non_zero_time[subject][segment][run][tool+'_y']
                fig = go.Figure()
                #for tool in tools_name:  

                a = np.array(optitrack_non_zero[subject][segment][run][tool+'_y'])
                b = np.array(interpolated_data[subject][segment][run][tool+'_y'])
                c = pos_y[subject][segment][run][tool]
                d = pos_y_tip[subject][segment][run][tool]
                e = pos_y_tip_outlier[subject][segment][run][tool]
                f = np.array(interpolated_data_tip_y[subject][segment][run][tool])
                g = pos_y_tip_filter[subject][segment][run][tool] 

                fig.add_trace(go.Scatter(y=b, x=time_vector_interp[subject][segment][run][tool+'_y']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name='interp_'+tool+'_y',marker=dict(color='LightSkyBlue',size=10)))

                fig.add_trace(go.Scatter(y=a, x=time_current-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name=tool+'_y raw',marker=dict(size=5)))

                #fig.add_trace(go.Scatter(y=a, x=time_vector_interp[subject][segment][run]['tweezer_y']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                #    mode='markers', name='tweezer_y',marker=dict(size=3) ))
                fig.add_trace(go.Scatter(y=c, x=time_current-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name=tool+'_y',marker=dict(size=3) ))
                #fig.add_trace(go.Scatter(y=b, x=time_vector_interp[subject][segment][run]['tweezer_y']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                #    mode='markers', name='position tip'))


                fig.add_trace(go.Scatter(y=d, x=time_current-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name='position tip', marker=dict(color='yellow',  size=15)))

                fig.add_trace(go.Scatter(y=f, x=time_vector_interp[subject][segment][run][tool+'_y']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name='position tip filter outliers interp', marker=dict(color='LightSkyBlue',size=10)))  


                fig.add_trace(go.Scatter(y=list(e[0]), x=time_current-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name='position tip filter outliers', marker=dict(color='violet', size=8)))

    
                fig.add_trace(go.Scatter(y=g, x=time_vector_interp[subject][segment][run][tool+'_y']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name='position tip filter median', marker=dict(color='red', size=3)))
        

                fig.update_layout(title='Subject: ' + subject + ', Run: ' + str(run) +', Segment:' + str(segment),
                    xaxis_title='Time',
                    yaxis_title='Optitrack data [m]',
                    width=1000,
                    height=500)
                #fig.show()

                if not os.path.exists(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK', 'POSITION_y', tool)):
                    os.makedirs(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK', 'POSITION_y', tool))
                fig.write_image(join('..', 'data_paper', 'PLOTS' , 'OPTITRACK',  'POSITION_y', tool, 'optitrack_position_subject_'+ subject+ 'run'+ str(run) + '_segment' + segment  +'.png'))

            

In [ ]:
# plots signaux total

for subject in subject_list:   
    print(subject)
    for segment in segment_list:
        for run in run_list[subject]:
            for tool in tools_name:
                time_current =  optitrack_non_zero_time[subject][segment][run][tool+'_x']
                fig = go.Figure()
                #for tool in tools_name:  

                c = np.array(optitrack_non_zero[subject][segment][run][tool+'_x'])
                d = np.array(interpolated_data[subject][segment][run][tool+'_x'])
                a = pos_x[subject][segment][run][tool]
                b = pos_x_tip[subject][segment][run][tool]
                e = pos_x_tip_outlier[subject][segment][run][tool]
                f = np.array(interpolated_data_tip_x[subject][segment][run][tool])
                g = pos_x_tip_filter[subject][segment][run][tool] 


                fig.add_trace(go.Scatter(y=d, x=time_vector_interp[subject][segment][run][tool+'_x']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name='interp' + tool + '_x',marker=dict(color='LightSkyBlue',size=10)))

                fig.add_trace(go.Scatter(y=c, x=time_current-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name=tool +'_x raw',marker=dict(color = 'navy', size=5)))

                #fig.add_trace(go.Scatter(y=a, x=time_vector_interp[subject][segment][run]['tweezer_y']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                #    mode='markers', name='tweezer_y',marker=dict(size=3) ))
                fig.add_trace(go.Scatter(y=a, x=time_current-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name=tool+'_x from transf',marker=dict(size=3) ))
                #fig.add_trace(go.Scatter(y=b, x=time_vector_interp[subject][segment][run]['tweezer_y']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                #    mode='markers', name='position tip'))

                fig.add_trace(go.Scatter(y=f, x=time_vector_interp[subject][segment][run][tool+'_x']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name='position tip interp', marker=dict(color='LightGreen',size=15)))

                fig.add_trace(go.Scatter(y=b, x=time_current-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name='position tip ', marker=dict(color='forestgreen', size=12)))
                
                fig.add_trace(go.Scatter(y=list(e[0]), x=time_current-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name='position tip filter outliers', marker=dict(color='indigo', size=8)))

                fig.add_trace(go.Scatter(y=g, x=time_vector_interp[subject][segment][run][tool+'_x']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name='position tip filter median', marker=dict(color='red', size=3)))
                


                fig.update_layout(title='Subject: ' + subject + ', Run: ' + str(run) +', Segment:' + str(segment),
                    xaxis_title='Time',
                    yaxis_title='Optitrack data [m]',
                    width=1000,
                    height=500)
                #fig.show()

                if not os.path.exists(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK', 'POSITION_x', tool)):
                    os.makedirs(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK',  'POSITION_x', tool))
                fig.write_image(join('..', 'data_paper', 'PLOTS' , 'OPTITRACK', 'POSITION_x', tool, 'x_optitrack_position_subject_'+ subject+ 'run'+ str(run) + '_segment' + segment  +'.png'))

         

In [112]:
# plots signaux total
for subject in subject_list:   
    for segment in segment_list:
        for run in run_list[subject]:
            for tool in tools_name:  
                time_current =  optitrack_non_zero_time[subject][segment][run][tool +'_z']
                fig = go.Figure()


                c = np.array(optitrack_non_zero[subject][segment][run][tool +'_z'])
                d = np.array(interpolated_data[subject][segment][run][tool +'_z'])
                a = pos_z[subject][segment][run][tool]
                b = pos_z_tip[subject][segment][run][tool]
                e = pos_z_tip_outlier[subject][segment][run][tool]
                f = np.array(interpolated_data_tip_z[subject][segment][run][tool])
                g = pos_z_tip_filter[subject][segment][run][tool] 

                fig.add_trace(go.Scatter(y=d, x=time_vector_interp[subject][segment][run][tool +'_z']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name='interp' + tool + '_z',marker=dict(color='LightSkyBlue',size=10)))

                fig.add_trace(go.Scatter(y=c, x=time_current-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name=tool + '_z raw',marker=dict(size=5)))

                #fig.add_trace(go.Scatter(y=a, x=time_vector_interp[subject][segment][run]['tweezer_y']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                #    mode='markers', name='tweezer_y',marker=dict(size=3) ))
                fig.add_trace(go.Scatter(y=a, x=time_current-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name=tool + '_z',marker=dict(size=3) ))
                #fig.add_trace(go.Scatter(y=b, x=time_vector_interp[subject][segment][run]['tweezer_y']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                #    mode='markers', name='position tip'))
                fig.add_trace(go.Scatter(y=f, x=time_vector_interp[subject][segment][run][tool +'_z']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name='position tip interp', marker=dict(color='LightGreen',size=12)))

                fig.add_trace(go.Scatter(y=b, x=time_current-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name='position tip', marker=dict(size=8)))

                fig.add_trace(go.Scatter(y=g, x=time_vector_interp[subject][segment][run][tool+'_z']-data_segments[subject][segment][run][sensor]['rel_time'][0],
                        mode='markers', name='position tip filter median', marker=dict(color='red', size=3)))
                
                 fig.add_trace(go.Scatter(y=list(e[0]), x=time_current-data_segments[subject][segment][run][sensor]['rel_time'][0],
                    mode='markers', name='position tip filter outliers', marker=dict(color='violet', size=8)))

                fig.update_layout(title='Subject: ' + subject + ', Run: ' + str(run) +', Segment:' + str(segment),
                    xaxis_title='Time',
                    yaxis_title='Optitrack data [m]',
                    width=1000,
                    height=500)
                #fig.show()

                if not os.path.exists(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK', 'POSITION_z', tool)):
                    os.makedirs(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK',  'POSITION_z', tool))
                fig.write_image(join('..', 'data_paper', 'PLOTS' , 'OPTITRACK',  'POSITION_z', tool, 'z_optitrack_position_subject_'+ subject+ 'run'+ str(run) + '_segment' + segment  +'.png'))

         

In [188]:
for subject in subject_list:   
    for segment in segment_list:
        for run in run_list[subject]:
            for tool in tools_name:  
                
                fig, axs = plt.subplots(1,3,figsize=(19.5, 4.5))
                x = time_vector_interp[subject][segment][run][tool+ '_' + 'x']-data_segments[subject][segment][run][sensor]['rel_time'][0]

                ax1 = axs[0]
                ax2 = axs[1]
                ax3 = axs[2]
                
                ax1.plot(x[1:-1], speed_x_tip[subject][segment][run][tool],color='lightgreen', label = 'Speed x base', linewidth =3 )
                ax1.plot(x, speed_x_tip_estimator[subject][segment][run][tool],color='green', label = 'Speed x est', linewidth =1 )

                ax2.plot(x[1:-1], speed_y_tip[subject][segment][run][tool],color='lightcoral', label = 'Speed y base', linewidth =3 )
                ax2.plot(x, speed_y_tip_estimator[subject][segment][run][tool] ,color='red', label = 'Speed y est', linewidth =1 )
                
                ax3.plot(x[1:-1], speed_z_tip[subject][segment][run][tool],color='blue', label = 'Speed z base', linewidth =3 )
                ax3.plot(x, speed_z_tip_estimator[subject][segment][run][tool],color='turquoise', label = 'Speed z est', linewidth =1)
                
                ax2.legend()
                ax1.legend()
                ax3.legend()

                ax1.grid()
                ax2.grid()
                ax3.grid()

                for ax in axs.flat:
                    ax.set(xlabel='time', ylabel='speed m/s')

                ax1.set_title("Speed x")
                ax2.set_title("Speed y") 
                ax3.set_title("Speed z")

                fig.suptitle("Subject_"+ subject + "Segment " + segment + " run: " + run + " tool: " + tool, x = 0.5, fontsize=15)    
                #fig.subplots_adjust(hspace=0.8, top=0.8)

                if not os.path.exists(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK', 'SPEED', tool)):
                    os.makedirs(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK',  'SPEED', tool))
                #fig.write_image(join('..', 'data_paper', 'PLOTS' , 'OPTITRACK',  'SPEED', tool, 'speed_optitrack_subject_'+ subject+ 'run'+ str(run) + '_segment' + segment  +'.png'))

                plt.savefig(join('..', 'data_paper', 'PLOTS' , 'OPTITRACK',  'SPEED', tool, 'speed_optitrack_subject_'+ subject+ 'run'+ str(run) + '_segment' + segment  +'.png'),bbox_inches='tight')
                plt.close(fig)

In [173]:
print(len(x[1:-2]))

3792


In [194]:
plt.ioff()
for subject in subject_list:   
    for segment in segment_list:
        for run in run_list[subject]:
            for tool in tools_name:  
                
              
                
                fig, axs = plt.subplots(1,3,figsize=(19.5, 4.5))
                x = time_vector_interp[subject][segment][run][tool+ '_' + 'x']

                ax1 = axs[0]
                ax2 = axs[1]
                ax3 = axs[2]
                
                ax1.plot(x[2:-2], acc_x_tip[subject][segment][run][tool],color='lightgreen', label = 'Acc x base', linewidth =3 )
                ax1.plot(x, acc_x_tip_estimator[subject][segment][run][tool],color='green', label = 'Acc x est', linewidth =1)
                ax2.plot(x[2:-2], acc_y_tip[subject][segment][run][tool],color='lightcoral', label = 'Acc y base', linewidth =3 )
                ax2.plot(x, acc_y_tip_estimator[subject][segment][run][tool] ,color='red', label = 'Acc y est', linewidth =1)
                ax3.plot(x[2:-2], acc_z_tip[subject][segment][run][tool],color='turquoise', label = 'Acc y base', linewidth =3 )
                ax3.plot(x, acc_z_tip_estimator[subject][segment][run][tool],color='blue', label = 'Acc z est', linewidth =1)

                ax2.legend()
                ax1.legend()
                ax3.legend()
                ax1.grid()
                ax2.grid()
                ax3.grid()

                for ax in axs.flat:
                    ax.set(xlabel='time', ylabel='acc m/s2')

                ax1.set_title("Acc x")
                ax2.set_title("Acc y") 
                ax3.set_title("Acc z")

                fig.suptitle("Subject: "+ subject + "; Segment " + segment + "; run: " + run + "; tool: " + tool, x = 0.5, fontsize=15)    
                #fig.subplots_adjust(hspace=0.8, top=0.8)

                if not os.path.exists(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK', 'ACC', tool)):
                    os.makedirs(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK',  'ACC', tool))
                #fig.write_image(join('..', 'data_paper', 'PLOTS' , 'OPTITRACK',  'SPEED', tool, 'speed_optitrack_subject_'+ subject+ 'run'+ str(run) + '_segment' + segment  +'.png'))

                plt.savefig(join('..', 'data_paper', 'PLOTS' , 'OPTITRACK',  'ACC', tool, 'acc_optitrack_subject_'+ subject+ 'run'+ str(run) + '_segment' + segment  +'.png'),bbox_inches='tight')
                plt.close(fig)

In [198]:

for subject in subject_list:   
    for segment in segment_list:
        for run in run_list[subject]:
            for tool in tools_name:  
                
                fig, axs = plt.subplots(1,3,figsize=(19.5, 4.5))
                x = time_vector_interp[subject][segment][run][tool+ '_' + 'x']

                ax1 = axs[0]
                ax2 = axs[1]
                ax3 = axs[2]
                
                ax1.plot(x[3:-3], jerk_x_tip[subject][segment][run][tool],color='lightgreen', label = 'jerk x base', linewidth =3 )
                ax1.plot(x, jerk_x_tip_estimator[subject][segment][run][tool],color='green', label = 'jerk x est', linewidth =1)
                ax2.plot(x[3:-3], jerk_x_tip[subject][segment][run][tool],color='lightcoral', label = 'jerk y base', linewidth =3 )
                ax2.plot(x, jerk_y_tip_estimator[subject][segment][run][tool] ,color='red', label = 'jerk y est', linewidth =1)
                ax3.plot(x[3:-3], jerk_z_tip[subject][segment][run][tool],color='turquoise', label = 'jerk z base', linewidth =3 )
                ax3.plot(x, jerk_z_tip_estimator[subject][segment][run][tool],color='blue', label = 'jerk z est', linewidth =1)

                ax1.grid()
                ax2.grid()
                ax3.grid()
                ax2.legend()
                ax1.legend()
                ax3.legend()

                for ax in axs.flat:
                    ax.set(xlabel='time', ylabel='jerk m/s3')

                ax1.set_title("Jerk x")
                ax2.set_title("Jerk y") 
                ax3.set_title("Jerk z")

                fig.suptitle("Subject_"+ subject + "Segment " + segment + " run: " + run + " tool: " + tool, x = 0.15, fontsize=15)    
                #fig.subplots_adjust(hspace=0.8, top=0.8)

                if not os.path.exists(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK', 'JERK', tool)):
                    os.makedirs(join(path_to_data_folder, 'PLOTS' , 'OPTITRACK',  'JERK', tool))
                #fig.write_image(join('..', 'data_paper', 'PLOTS' , 'OPTITRACK',  'SPEED', tool, 'speed_optitrack_subject_'+ subject+ 'run'+ str(run) + '_segment' + segment  +'.png'))

                plt.savefig(join('..', 'data_paper', 'PLOTS' , 'OPTITRACK',  'JERK', tool, 'jerk_optitrack_subject_'+ subject+ 'run'+ str(run) + '_segment' + segment  +'.png'),bbox_inches='tight')
                plt.close(fig)

## METRICS

In [163]:
print(jerk_x_tip_estimator[subject][segment][run][tool])

[0. 0. 0. ... 0. 0. 0.]


# DATA TIME

## MT1: completion time

In [15]:
segment_new = ['2','3','4','5','6','1']

duration_run_segments = {}
mean_duration_run_segments ={}
std_duration_run_segments ={}

for subject in subject_list:
    duration_run_segments[subject] = {} 
    mean_duration_run_segments[subject] = {} 
    std_duration_run_segments[subject] = {}  
    for segment in segment_list:
        duration_run_segments[subject][segment] = [] 
        mean_duration_run_segments[subject][segment] = []
        std_duration_run_segments[subject][segment] = []
        
        
for subject in subject_list:
    for segment in segment_list:
        for run in run_list[subject]: 
            duration_run_segments[subject][segment].append(np.array(data_segments[subject][segment][run]['optitrack']['rel_time'])[-1] - np.array(data_segments[subject][segment][run]['optitrack']['rel_time'])[0] )
        
        mean_duration_run_segments[subject][segment] = np.mean(duration_run_segments[subject][segment])
        std_duration_run_segments[subject][segment] = np.std(duration_run_segments[subject][segment])


## MT2: Idle time (need v)

In [125]:
subject_list = ['S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12']


## MT3: act percentage considering tim